In [3]:
import pandas as pd
from pprintpp import pprint as pp
import numpy as np 
import altair as alt
from vega_datasets import data
import random
pd.set_option('display.max_rows', 500)
import csv
from functools import reduce
import simplejson as json



/Users/shaneburke/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# March 2020 vs. March 2019

In [4]:
mar19 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190302.txt')
mar20 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_200307.txt')

In [5]:
mar19['change'] = mar19.ENTRIES.diff()
mar20['change'] = mar20.ENTRIES.diff()

In [6]:
mar19['full'] = mar19['STATION'] + " " + mar19['DIVISION']
mar20['full'] = mar20['STATION'] + " " + mar20['DIVISION']

In [7]:
mar19 = mar19[(mar19['change'] < 10000) & (mar19['change'] > 0 )]
mar20 = mar20[(mar20['change'] < 10000) & (mar20['change'] > 0 )]

In [8]:
mar19_week = mar19.groupby(['full']).sum().reset_index()
mar20_week = mar20.groupby(['full']).sum().reset_index()

In [9]:
mar20

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,change,full
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/29/2020,07:00:00,REGULAR,7394755,2508857,8.0,59 ST BMT
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/29/2020,11:00:00,REGULAR,7394829,2508945,74.0,59 ST BMT
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/29/2020,15:00:00,REGULAR,7395039,2509013,210.0,59 ST BMT
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/29/2020,19:00:00,REGULAR,7395332,2509058,293.0,59 ST BMT
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/29/2020,23:00:00,REGULAR,7395518,2509104,186.0,59 ST BMT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
207798,TRAM2,R469,00-03-01,RIT-ROOSEVELT,R,RIT,03/06/2020,20:00:00,REGULAR,3129617,395948,49.0,RIT-ROOSEVELT RIT
207820,TRAM2,R469,00-05-00,RIT-ROOSEVELT,R,RIT,03/03/2020,12:00:00,REGULAR,3072,62,11.0,RIT-ROOSEVELT RIT
207827,TRAM2,R469,00-05-00,RIT-ROOSEVELT,R,RIT,03/04/2020,16:00:00,REGULAR,3073,62,1.0,RIT-ROOSEVELT RIT
207833,TRAM2,R469,00-05-00,RIT-ROOSEVELT,R,RIT,03/05/2020,16:00:00,REGULAR,3074,62,1.0,RIT-ROOSEVELT RIT


In [10]:
mar19v20 = mar19_week.merge(mar20_week, on='full', suffixes=("_before", "_after"))

In [11]:
mar19v20
mar19v20['month'] = "March 2020"
mar19v20['properties.group_id'] = 0

In [12]:
mar19v20['diff'] = mar19v20['change_before'] - mar19v20['change_after']

In [13]:
mar19v20['pct_change'] = mar19v20['diff'] / mar19v20['change_before'] * 100

In [14]:
mar19v20

,full,ENTRIES_before,EXITS _before,change_before,ENTRIES_after,EXITS _after,change_after,month,properties.group_id,diff,pct_change
0,1 AV BMT,40922000340,42321806903,100057.0,20263225,32108845,89166.0,March 2020,0,10891.0,10.884796
1,103 ST IND,1210818910,1044619647,26013.0,1274444327,1089595810,27535.0,March 2020,0,-1522.0,-5.850921
2,103 ST IRT,5161376898,23111135664,159007.0,4234576698,3711613609,148898.0,March 2020,0,10109.0,6.357582
3,103 ST-CORONA IRT,2861794426,2889710030,123534.0,3135782288,3099567684,126589.0,March 2020,0,-3055.0,-2.473003
4,104 ST BMT,44162583658,74786858631,15727.0,12708562,6574792,15531.0,March 2020,0,196.0,1.246264
5,104 ST IND,357087937,413478147,9110.0,373645975,412412454,9017.0,March 2020,0,93.0,1.020856
6,110 ST IRT,1424259519,1422297329,66236.0,1657866056,1582026181,63907.0,March 2020,0,2329.0,3.516215
7,111 ST IND,312240123,169253811,15740.0,460068197,150341404,14965.0,March 2020,0,775.0,4.923761
8,111 ST IRT,2104001394,1281138138,68827.0,2248483324,1358747469,104141.0,March 2020,0,-35314.0,-51.308353
9,116 ST IND,11837348803,12760677250,41760.0,14934327406,16100711550,43955.0,March 2020,0,-2195.0,-5.256226


# April 2020 vs. April 2019

In [15]:
apr19 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190406.txt')
apr20 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_200404.txt')

In [16]:
apr19['change'] = apr19.ENTRIES.diff()
apr20['change'] = apr20.ENTRIES.diff()

In [17]:
apr19['full'] = apr19['STATION'] + " " + apr19['DIVISION']
apr20['full'] = apr20['STATION'] + " " + apr20['DIVISION']

In [18]:
apr19 = apr19[(apr19['change'] < 10000) & (apr19['change'] > 0 )]
apr20 = apr20[(apr20['change'] < 10000) & (apr20['change'] > 0 )]

In [19]:
apr19_week = apr19.groupby(['full']).sum().reset_index()
apr20_week = apr20.groupby(['full']).sum().reset_index()

In [20]:
apr19v20 = apr19_week.merge(apr20_week, on='full', suffixes=("_before", "_after"))

In [21]:
apr19v20['diff'] = apr19v20['change_after'] - apr19v20['change_before']

In [22]:
apr19v20['pct_change'] = apr19v20['diff'] / apr19v20['change_before'] * 100

In [26]:
apr19v20
apr19v20['month'] = "April 2020"
apr19v20['properties.group_id'] = 1

In [25]:
apr19v20.head()

,full,ENTRIES_before,EXITS _before,change_before,ENTRIES_after,EXITS _after,change_after,diff,pct_change,month,properties.group_id
0,1 AV BMT,54109720356,56009034723,128577.0,27416408,40421634,15461.0,-113116.0,-87.975299,April 2020,1
1,103 ST IND,1216554406,1048654097,29283.0,1245601211,1064706763,3139.0,-26144.0,-89.280470,April 2020,1
2,103 ST IRT,5397710715,22836702353,152007.0,4204818390,3699613850,21914.0,-130093.0,-85.583559,April 2020,1
3,103 ST-CORONA IRT,2891197623,2916399072,125705.0,2522861614,2577188519,17204.0,-108501.0,-86.313989,April 2020,1
4,104 ST BMT,46408103182,78602493883,16472.0,13374641,6903801,3630.0,-12842.0,-77.962603,April 2020,1


# May 2020 vs. May 2019

In [28]:
may19 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190504.txt')
may20 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_200502.txt')

In [42]:
may19['change'] = may19.ENTRIES.diff()
may20['change'] = may20.ENTRIES.diff()

may19['full'] = may19['STATION'] + " " + may19['DIVISION']
may20['full'] = may20['STATION'] + " " + may20['DIVISION']

may19 = may19[(may19['change'] < 10000) & (may19['change'] > 0 )]
may20 = may20[(may20['change'] < 10000) & (may20['change'] > 0 )]

may19_week = may19.groupby(['full']).sum().reset_index()
may20_week = may20.groupby(['full']).sum().reset_index()

may19v20 = may19_week.merge(may20_week, on='full', suffixes=("_before", "_after"))

may19v20['diff'] = may19v20['change_after'] - may19v20['change_before']

may19v20['pct_change'] = may19v20['diff'] / may19v20['change_before'] * 100

may19v20
may19v20['month'] = "May 2020"
may19v20['properties.group_id'] = 2

In [30]:
may19v20.head()

,full,ENTRIES_before,EXITS _before,change_before,ENTRIES_after,EXITS _after,change_after,diff,pct_change,month,properties.group_id
0,1 AV BMT,53823158573,56115161232,103989.0,19246997825,20562295985,16156.0,-87833.0,-84.463741,May 2020,2
1,103 ST IND,1221400634,1052023871,30984.0,1227955608,1046900126,2665.0,-28319.0,-91.398786,May 2020,2
2,103 ST IRT,6024372266,24798991449,163546.0,3708424466,3293563635,19512.0,-144034.0,-88.069412,May 2020,2
3,103 ST-CORONA IRT,3042542571,3071395441,125882.0,2520348362,2571225477,18532.0,-107350.0,-85.278276,May 2020,2
4,104 ST BMT,46410519592,78603084105,16472.0,12912248,6705894,3925.0,-12547.0,-76.171685,May 2020,2


# June 2020 vs. June 2019

In [31]:
jun19 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190601.txt')
jun20 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_200606.txt')

In [32]:
jun19['change'] = jun19.ENTRIES.diff()
jun20['change'] = jun20.ENTRIES.diff()

jun19['full'] = jun19['STATION'] + " " + jun19['DIVISION']
jun20['full'] = jun20['STATION'] + " " + jun20['DIVISION']

jun19 = jun19[(jun19['change'] < 10000) & (jun19['change'] > 0 )]
jun20 = jun20[(jun20['change'] < 10000) & (jun20['change'] > 0 )]

jun19_week = jun19.groupby(['full']).sum().reset_index()
jun20_week = jun20.groupby(['full']).sum().reset_index()

june19v20 = jun19_week.merge(jun20_week, on='full', suffixes=("_before", "_after"))

june19v20['diff'] = june19v20['change_after'] - june19v20['change_before']

june19v20['pct_change'] = june19v20['diff'] / june19v20['change_before'] * 100

june19v20
june19v20['month'] = "June 2020"
june19v20['properties.group_id'] = 3

In [33]:
june19v20.head()

,full,ENTRIES_before,EXITS _before,change_before,ENTRIES_after,EXITS _after,change_after,diff,pct_change,month,properties.group_id
0,1 AV BMT,47966099023,50261153124,88542.0,497826455,1127230061,21530.0,-67012.0,-75.683856,June 2020,3
1,103 ST IND,1226511562,1055505963,28272.0,1150644621,967838432,3761.0,-24511.0,-86.697085,June 2020,3
2,103 ST IRT,5808778202,23542650144,145317.0,3613949023,3073365433,25850.0,-119467.0,-82.211304,June 2020,3
3,103 ST-CORONA IRT,2932408570,2946613062,119682.0,2335734650,2343975412,32513.0,-87169.0,-72.833843,June 2020,3
4,104 ST BMT,28309807803,47938562924,11092.0,13028087,6817297,4893.0,-6199.0,-55.887126,June 2020,3


# July 2020 vs. July 2019

In [35]:
jul19 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190706.txt')
jul20 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_200704.txt')

In [36]:
jul19['change'] = jul19.ENTRIES.diff()
jul20['change'] = jul20.ENTRIES.diff()

jul19['full'] = jul19['STATION'] + " " + jul19['DIVISION']
jul20['full'] = jul20['STATION'] + " " + jul20['DIVISION']

jul19 = jul19[(jul19['change'] < 10000) & (jul19['change'] > 0 )]
jul20 = jul20[(jul20['change'] < 10000) & (jul20['change'] > 0 )]

jul19_week = jul19.groupby(['full']).sum().reset_index()
jul20_week = jul20.groupby(['full']).sum().reset_index()

jul19v20 = jul19_week.merge(jul20_week, on='full', suffixes=("_before", "_after"))

jul19v20['diff'] = jul19v20['change_after'] - jul19v20['change_before']

jul19v20['pct_change'] = jul19v20['diff'] / jul19v20['change_before'] * 100

jul19v20
jul19v20['month'] = "July 2020"
jul19v20['properties.group_id'] = 4

# August 2020 vs. August 2019

In [37]:
aug19 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190803.txt')
aug20 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_200801.txt')

In [43]:
aug19['change'] = aug19.ENTRIES.diff()
aug20['change'] = aug20.ENTRIES.diff()

aug19['full'] = aug19['STATION'] + " " + aug19['DIVISION']
aug20['full'] = aug20['STATION'] + " " + aug20['DIVISION']

aug19 = aug19[(aug19['change'] < 10000) & (aug19['change'] > 0 )]
aug20 = aug20[(aug20['change'] < 10000) & (aug20['change'] > 0 )]

aug19_week = aug19.groupby(['full']).sum().reset_index()
aug20_week = aug20.groupby(['full']).sum().reset_index()

aug19v20 = aug19_week.merge(aug20_week, on='full', suffixes=("_before", "_after"))

aug19v20['diff'] = aug19v20['change_after'] - aug19v20['change_before']

aug19v20['pct_change'] = aug19v20['diff'] / aug19v20['change_before'] * 100

aug19v20
aug19v20['month'] = "August 2020"
aug19v20['properties.group_id'] = 5

# September 2020 vs. September 2019

In [39]:
sep19 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190907.txt')
sep20 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_200905.txt')

In [44]:
sep19['change'] = sep19.ENTRIES.diff()
sep20['change'] = sep20.ENTRIES.diff()

sep19['full'] = sep19['STATION'] + " " + sep19['DIVISION']
sep20['full'] = sep20['STATION'] + " " + sep20['DIVISION']

sep19 = sep19[(sep19['change'] < 10000) & (sep19['change'] > 0 )]
sep20 = sep20[(sep20['change'] < 10000) & (sep20['change'] > 0 )]

sep19_week = sep19.groupby(['full']).sum().reset_index()
sep20_week = sep20.groupby(['full']).sum().reset_index()

sep19v20 = sep19_week.merge(sep20_week, on='full', suffixes=("_before", "_after"))

sep19v20['diff'] = sep19v20['change_after'] - sep19v20['change_before']

sep19v20['pct_change'] = sep19v20['diff'] / sep19v20['change_before'] * 100

sep19v20['month'] = "September 2020"
sep19v20['properties.group_id'] = 6

# October 2020 vs. October 2019

In [46]:
oct19 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_191005.txt')
oct20 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_201003.txt')

In [47]:
oct19['change'] = oct19.ENTRIES.diff()
oct20['change'] = oct20.ENTRIES.diff()

oct19['full'] = oct19['STATION'] + " " + oct19['DIVISION']
oct20['full'] = oct20['STATION'] + " " + oct20['DIVISION']

oct19 = oct19[(oct19['change'] < 10000) & (oct19['change'] > 0 )]
oct20 = oct20[(oct20['change'] < 10000) & (oct20['change'] > 0 )]

oct19_week = oct19.groupby(['full']).sum().reset_index()
oct20_week = oct20.groupby(['full']).sum().reset_index()

oct19v20 = oct19_week.merge(oct20_week, on='full', suffixes=("_before", "_after"))

oct19v20['diff'] = oct19v20['change_after'] - oct19v20['change_before']

oct19v20['pct_change'] = oct19v20['diff'] / oct19v20['change_before'] * 100

oct19v20
oct19v20['month'] = "Oct 2020"
oct19v20['properties.group_id'] = 7


# November 2020 vs. November 2019

In [48]:
nov19 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_191102.txt')
nov20 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_201107.txt')

In [49]:
nov19['change'] = nov19.ENTRIES.diff()
nov20['change'] = nov20.ENTRIES.diff()

nov19['full'] = nov19['STATION'] + " " + nov19['DIVISION']
nov20['full'] = nov20['STATION'] + " " + nov20['DIVISION']

nov19 = nov19[(nov19['change'] < 10000) & (nov19['change'] > 0 )]
nov20 = nov20[(nov20['change'] < 10000) & (nov20['change'] > 0 )]

nov19_week = nov19.groupby(['full']).sum().reset_index()
nov20_week = nov20.groupby(['full']).sum().reset_index()

nov19v20 = nov19_week.merge(nov20_week, on='full', suffixes=("_before", "_after"))

nov19v20['diff'] = nov19v20['change_after'] - nov19v20['change_before']

nov19v20['pct_change'] = nov19v20['diff'] / nov19v20['change_before'] * 100


In [118]:

nov19v20
nov19v20['month'] = "Nov 2020"
nov19v20['properties.group_id'] = 8



# December 2020 vs. December 2019

In [51]:
dec19 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_191207.txt')
dec20 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_201205.txt')

In [52]:
dec19['change'] = dec19.ENTRIES.diff()
dec20['change'] = dec20.ENTRIES.diff()

dec19['full'] = dec19['STATION'] + " " + dec19['DIVISION']
dec20['full'] = dec20['STATION'] + " " + dec20['DIVISION']

dec19 = dec19[(dec19['change'] < 10000) & (dec19['change'] > 0 )]
dec20 = dec20[(dec20['change'] < 10000) & (dec20['change'] > 0 )]

dec19_week = dec19.groupby(['full']).sum().reset_index()
dec20_week = dec20.groupby(['full']).sum().reset_index()

dec19v20 = dec19_week.merge(dec20_week, on='full', suffixes=("_before", "_after"))

dec19v20['diff'] = dec19v20['change_after'] - dec19v20['change_before']

dec19v20['pct_change'] = dec19v20['diff'] / dec19v20['change_before'] * 100

dec19v20
dec19v20['month'] = "Dec 2020"
dec19v20['properties.group_id'] = 9


# January 2021 vs. January 2020

In [53]:
jan20 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_200104.txt')
jan21 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_210102.txt')

In [54]:
jan20['change'] = jan20.ENTRIES.diff()
jan21['change'] = jan21.ENTRIES.diff()

jan20['full'] = jan20['STATION'] + " " + jan20['DIVISION']
jan21['full'] = jan21['STATION'] + " " + jan21['DIVISION']

jan20 = jan20[(jan20['change'] < 10000) & (jan20['change'] > 0 )]
jan21 = jan21[(jan21['change'] < 10000) & (jan21['change'] > 0 )]

jan20_week = jan20.groupby(['full']).sum().reset_index()
jan21_week = jan21.groupby(['full']).sum().reset_index()

jan20v21 = jan20_week.merge(jan21_week, on='full', suffixes=("_before", "_after"))

jan20v21['diff'] = jan20v21['change_after'] - jan20v21['change_before']

jan20v21['pct_change'] = jan20v21['diff'] / jan20v21['change_before'] * 100

jan20v21
jan20v21['month'] = "Jan 2021"
jan20v21['properties.group_id'] = 10



# February 2021 vs. February 2020

In [55]:
feb20 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_200201.txt')
feb21 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_210206.txt')

In [56]:
feb20['change'] = feb20.ENTRIES.diff()
feb21['change'] = feb21.ENTRIES.diff()

feb20['full'] = feb20['STATION'] + " " + feb20['DIVISION']
feb21['full'] = feb21['STATION'] + " " + feb21['DIVISION']

feb20 = feb20[(feb20['change'] < 10000) & (feb20['change'] > 0 )]
feb21 = feb21[(feb21['change'] < 10000) & (feb21['change'] > 0 )]

feb20_week = feb20.groupby(['full']).sum().reset_index()
feb21_week = feb21.groupby(['full']).sum().reset_index()

feb20v21 = feb20_week.merge(feb21_week, on='full', suffixes=("_before", "_after"))

feb20v21['diff'] = feb20v21['change_after'] - feb20v21['change_before']

feb20v21['pct_change'] = feb20v21['diff'] / feb20v21['change_before'] * 100

feb20v21
feb20v21['month'] = "Feb 2021"
feb20v21['properties.group_id'] = 11



# March 2021 vs. March 2020

In [57]:
mar21 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_210306.txt')

In [58]:
mar21['change'] = mar21.ENTRIES.diff()
mar21['full'] = mar21['STATION'] + " " + mar21['DIVISION']

In [59]:
mar21 = mar21[(mar21['change'] < 10000) & (mar21['change'] > 0 )]

In [60]:
mar21_week = mar21.groupby(['full']).sum().reset_index()

In [61]:
mar20v21 = mar20_week.merge(mar21_week, on='full', suffixes=("_before", "_after"))

In [62]:
mar20v21['diff'] = mar20v21['change_after'] - mar20v21['change_before']

In [63]:
mar20v21['pct_change'] = mar20v21['diff'] / mar20v21['change_before'] * 100

In [117]:
mar20v21 
mar20v21['month'] = "Mar 2021"
mar20v21['properties.group_id'] = 12



# April 2021 vs. April 2019


In [68]:
apr21 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_210403.txt')

In [69]:
apr21['change'] = apr21.ENTRIES.diff()
apr21['full'] = apr21['STATION'] + " " + apr21['DIVISION']

apr21 = apr21[(apr21['change'] < 10000) & (apr21['change'] > 0 )]

apr21_week = apr21.groupby(['full']).sum().reset_index()

apr19v21 = apr19_week.merge(apr21_week, on='full', suffixes=("_before", "_after"))

apr19v21['diff'] = apr19v21['change_after'] - apr19v21['change_before']

apr19v21['pct_change'] = apr19v21['diff'] / apr19v21['change_before'] * 100

apr19v21 
apr19v21['month'] = "Apr 2021"
apr19v21['properties.group_id'] = 13



# May 2021 vs. May 2020

In [67]:
may21 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_210501.txt')

In [70]:
may21['change'] = may21.ENTRIES.diff()
may21['full'] = may21['STATION'] + " " + may21['DIVISION']

may21 = may21[(may21['change'] < 10000) & (may21['change'] > 0 )]

may21_week = may21.groupby(['full']).sum().reset_index()

may19v21 = may19_week.merge(may21_week, on='full', suffixes=("_before", "_after"))

may19v21['diff'] = may19v21['change_after'] - may19v21['change_before']

may19v21['pct_change'] = may19v21['diff'] / may19v21['change_before'] * 100

may19v21 
may19v21['month'] = "May 2021"
may19v21['properties.group_id'] = 14


# Combining dataframes

In [210]:
dataframes = [mar19v20, apr19v20, may19v20, june19v20, jul19v20, aug19v20, sep19v20, oct19v20, nov19v20, dec19v20, jan20v21, feb20v21, mar20v21, apr19v21, may19v21]

In [211]:
df = pd.concat(dataframes)

In [212]:
df.shape

(6707, 11)

In [213]:
df.head()

,full,ENTRIES_before,EXITS _before,change_before,ENTRIES_after,EXITS _after,change_after,month,properties.group_id,diff,pct_change
0,1 AV BMT,40922000340,42321806903,100057.0,20263225,32108845,89166.0,March 2020,0,10891.0,10.884796
1,103 ST IND,1210818910,1044619647,26013.0,1274444327,1089595810,27535.0,March 2020,0,-1522.0,-5.850921
2,103 ST IRT,5161376898,23111135664,159007.0,4234576698,3711613609,148898.0,March 2020,0,10109.0,6.357582
3,103 ST-CORONA IRT,2861794426,2889710030,123534.0,3135782288,3099567684,126589.0,March 2020,0,-3055.0,-2.473003
4,104 ST BMT,44162583658,74786858631,15727.0,12708562,6574792,15531.0,March 2020,0,196.0,1.246264


In [214]:
df = df[['full', 'month', 'properties.group_id', 'diff', 'pct_change']]

In [215]:
df.to_csv("subway_concat.csv")

In [216]:
df

,full,month,properties.group_id,diff,pct_change
0,1 AV BMT,March 2020,0,10891.0,10.884796
1,103 ST IND,March 2020,0,-1522.0,-5.850921
2,103 ST IRT,March 2020,0,10109.0,6.357582
3,103 ST-CORONA IRT,March 2020,0,-3055.0,-2.473003
4,104 ST BMT,March 2020,0,196.0,1.246264
...,...,...,...,...,...
441,WOODLAWN IRT,May 2021,14,-22403.0,-53.012305
442,WORLD TRADE CTR IND,May 2021,14,-93670.0,-86.670491
443,WTC-CORTLANDT IRT,May 2021,14,-70906.0,-78.456670
444,YORK ST IND,May 2021,14,-59661.0,-76.938254


# Adding location

In [125]:
locations = pd.read_csv("station_locations.csv")

In [126]:
df = df.merge(locations)

In [127]:
df2 = df 
#For backup

In [188]:
df['pct_change'] = round(df['pct_change'], 1)

In [193]:
df['pct_change_string'] = df['pct_change'].astype(str) + "%"

In [194]:
df['type'] = "Feature"

#Geometry Features
df['geometry.type'] = "Point"
df['geometry.coordinates'] = "-"

#Properties
df['properties.headline'] = df['full']+", "+df['month']+": "+df['pct_change_string']
df['properties.article'] = df['full']+df['month']
df['properties.radius'] = "5"
df['properties.name'] = df['full']
df['properties.color'] = "-"

df['properties.group_name'] = df['month']

In [195]:
df

,level_0,index,full,month,properties.group_id,diff,pct_change,latitude,longitude,type,...,geometry.coordinates,properties.headline,properties.article,properties.radius,properties.name,properties.color,properties.group_name,id,groupname_placeholder,pct_change_string
0,0,0,1 AV BMT,March 2020,0,10891.0,10.9,40.730953,-73.981628,Feature,...,-,"1 AV BMT, March 2020: 10.9%",1 AV BMTMarch 2020,5,1 AV BMT,-,March 2020,1,March 2020,10.9%
1,1,1,1 AV BMT,April 2020,1,-113116.0,-88.0,40.730953,-73.981628,Feature,...,-,"1 AV BMT, April 2020: -88.0%",1 AV BMTApril 2020,5,1 AV BMT,-,April 2020,2,April 2020,-88.0%
2,2,2,1 AV BMT,May 2020,2,-86863.0,-84.4,40.730953,-73.981628,Feature,...,-,"1 AV BMT, May 2020: -84.4%",1 AV BMTMay 2020,5,1 AV BMT,-,May 2020,3,May 2020,-84.4%
3,3,3,1 AV BMT,June 2020,3,-67012.0,-75.7,40.730953,-73.981628,Feature,...,-,"1 AV BMT, June 2020: -75.7%",1 AV BMTJune 2020,5,1 AV BMT,-,June 2020,4,June 2020,-75.7%
4,4,4,1 AV BMT,July 2020,4,-61145.0,-65.9,40.730953,-73.981628,Feature,...,-,"1 AV BMT, July 2020: -65.9%",1 AV BMTJuly 2020,5,1 AV BMT,-,July 2020,5,July 2020,-65.9%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4205,4205,4205,ZEREGA AV IRT,Jan 2021,10,-7209.0,-62.2,40.836488,-73.847036,Feature,...,-,"ZEREGA AV IRT, Jan 2021: -62.2%",ZEREGA AV IRTJan 2021,5,ZEREGA AV IRT,-,Jan 2021,4206,Jan 2021,-62.2%
4206,4206,4206,ZEREGA AV IRT,Feb 2021,11,-11151.0,-70.3,40.836488,-73.847036,Feature,...,-,"ZEREGA AV IRT, Feb 2021: -70.3%",ZEREGA AV IRTFeb 2021,5,ZEREGA AV IRT,-,Feb 2021,4207,Feb 2021,-70.3%
4207,4207,4207,ZEREGA AV IRT,Mar 2021,12,-9783.0,-62.0,40.836488,-73.847036,Feature,...,-,"ZEREGA AV IRT, Mar 2021: -62.0%",ZEREGA AV IRTMar 2021,5,ZEREGA AV IRT,-,Mar 2021,4208,Mar 2021,-62.0%
4208,4208,4208,ZEREGA AV IRT,Apr 2021,13,-10349.0,-63.4,40.836488,-73.847036,Feature,...,-,"ZEREGA AV IRT, Apr 2021: -63.4%",ZEREGA AV IRTApr 2021,5,ZEREGA AV IRT,-,Apr 2021,4209,Apr 2021,-63.4%


In [197]:
df

,level_0,index,full,month,properties.group_id,diff,pct_change,latitude,longitude,type,...,geometry.coordinates,properties.headline,properties.article,properties.radius,properties.name,properties.color,properties.group_name,id,groupname_placeholder,pct_change_string
0,0,0,1 AV BMT,March 2020,0,10891.0,10.9,40.730953,-73.981628,Feature,...,-,"1 AV BMT, March 2020: 10.9%",1 AV BMTMarch 2020,5,1 AV BMT,-,March 2020,1,March 2020,10.9%
1,1,1,1 AV BMT,April 2020,1,-113116.0,-88.0,40.730953,-73.981628,Feature,...,-,"1 AV BMT, April 2020: -88.0%",1 AV BMTApril 2020,5,1 AV BMT,-,April 2020,2,April 2020,-88.0%
2,2,2,1 AV BMT,May 2020,2,-86863.0,-84.4,40.730953,-73.981628,Feature,...,-,"1 AV BMT, May 2020: -84.4%",1 AV BMTMay 2020,5,1 AV BMT,-,May 2020,3,May 2020,-84.4%
3,3,3,1 AV BMT,June 2020,3,-67012.0,-75.7,40.730953,-73.981628,Feature,...,-,"1 AV BMT, June 2020: -75.7%",1 AV BMTJune 2020,5,1 AV BMT,-,June 2020,4,June 2020,-75.7%
4,4,4,1 AV BMT,July 2020,4,-61145.0,-65.9,40.730953,-73.981628,Feature,...,-,"1 AV BMT, July 2020: -65.9%",1 AV BMTJuly 2020,5,1 AV BMT,-,July 2020,5,July 2020,-65.9%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4205,4205,4205,ZEREGA AV IRT,Jan 2021,10,-7209.0,-62.2,40.836488,-73.847036,Feature,...,-,"ZEREGA AV IRT, Jan 2021: -62.2%",ZEREGA AV IRTJan 2021,5,ZEREGA AV IRT,-,Jan 2021,4206,Jan 2021,-62.2%
4206,4206,4206,ZEREGA AV IRT,Feb 2021,11,-11151.0,-70.3,40.836488,-73.847036,Feature,...,-,"ZEREGA AV IRT, Feb 2021: -70.3%",ZEREGA AV IRTFeb 2021,5,ZEREGA AV IRT,-,Feb 2021,4207,Feb 2021,-70.3%
4207,4207,4207,ZEREGA AV IRT,Mar 2021,12,-9783.0,-62.0,40.836488,-73.847036,Feature,...,-,"ZEREGA AV IRT, Mar 2021: -62.0%",ZEREGA AV IRTMar 2021,5,ZEREGA AV IRT,-,Mar 2021,4208,Mar 2021,-62.0%
4208,4208,4208,ZEREGA AV IRT,Apr 2021,13,-10349.0,-63.4,40.836488,-73.847036,Feature,...,-,"ZEREGA AV IRT, Apr 2021: -63.4%",ZEREGA AV IRTApr 2021,5,ZEREGA AV IRT,-,Apr 2021,4209,Apr 2021,-63.4%


In [150]:
df3 = df
# more saving

In [198]:
#To add in coordinates in list format, which JSON needs.
def get_coords(row):
    row['geometry.coordinates'] = [row['longitude'], row['latitude']]
    return row

df = df.apply(get_coords, axis=1)

In [199]:
def color(pct_change):
    
    if pct_change > 0 : 
        return "#66cb4e"
     
    if pct_change > -10 :
        return "#a1df61"
    
    if pct_change > -20 :
        return "#c6ec85"
    
    if pct_change > -30 : 
        return "#ecfaa7"
    
    if pct_change > -40 :
        return "#fff0a6"
    
    if pct_change > -50 :
        return "#ffd280"
    
    if pct_change > -60 : 
        return "#ffb25a"
    
    if pct_change > -70 :
        return "#ff7e3d"
        
    if pct_change > -80 :
        return "#f83f24"
    
    else: 
        return "#eb0000"

In [200]:
df['properties.color'] = df['pct_change'].map(color)

In [201]:
df

,level_0,index,full,month,properties.group_id,diff,pct_change,latitude,longitude,type,...,geometry.coordinates,properties.headline,properties.article,properties.radius,properties.name,properties.color,properties.group_name,id,groupname_placeholder,pct_change_string
0,0,0,1 AV BMT,March 2020,0,10891.0,10.9,40.730953,-73.981628,Feature,...,"[-73.981628, 40.730953]","1 AV BMT, March 2020: 10.9%",1 AV BMTMarch 2020,5,1 AV BMT,#66cb4e,March 2020,1,March 2020,10.9%
1,1,1,1 AV BMT,April 2020,1,-113116.0,-88.0,40.730953,-73.981628,Feature,...,"[-73.981628, 40.730953]","1 AV BMT, April 2020: -88.0%",1 AV BMTApril 2020,5,1 AV BMT,#eb0000,April 2020,2,April 2020,-88.0%
2,2,2,1 AV BMT,May 2020,2,-86863.0,-84.4,40.730953,-73.981628,Feature,...,"[-73.981628, 40.730953]","1 AV BMT, May 2020: -84.4%",1 AV BMTMay 2020,5,1 AV BMT,#eb0000,May 2020,3,May 2020,-84.4%
3,3,3,1 AV BMT,June 2020,3,-67012.0,-75.7,40.730953,-73.981628,Feature,...,"[-73.981628, 40.730953]","1 AV BMT, June 2020: -75.7%",1 AV BMTJune 2020,5,1 AV BMT,#f83f24,June 2020,4,June 2020,-75.7%
4,4,4,1 AV BMT,July 2020,4,-61145.0,-65.9,40.730953,-73.981628,Feature,...,"[-73.981628, 40.730953]","1 AV BMT, July 2020: -65.9%",1 AV BMTJuly 2020,5,1 AV BMT,#ff7e3d,July 2020,5,July 2020,-65.9%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4205,4205,4205,ZEREGA AV IRT,Jan 2021,10,-7209.0,-62.2,40.836488,-73.847036,Feature,...,"[-73.847036, 40.836488]","ZEREGA AV IRT, Jan 2021: -62.2%",ZEREGA AV IRTJan 2021,5,ZEREGA AV IRT,#ff7e3d,Jan 2021,4206,Jan 2021,-62.2%
4206,4206,4206,ZEREGA AV IRT,Feb 2021,11,-11151.0,-70.3,40.836488,-73.847036,Feature,...,"[-73.847036, 40.836488]","ZEREGA AV IRT, Feb 2021: -70.3%",ZEREGA AV IRTFeb 2021,5,ZEREGA AV IRT,#f83f24,Feb 2021,4207,Feb 2021,-70.3%
4207,4207,4207,ZEREGA AV IRT,Mar 2021,12,-9783.0,-62.0,40.836488,-73.847036,Feature,...,"[-73.847036, 40.836488]","ZEREGA AV IRT, Mar 2021: -62.0%",ZEREGA AV IRTMar 2021,5,ZEREGA AV IRT,#ff7e3d,Mar 2021,4208,Mar 2021,-62.0%
4208,4208,4208,ZEREGA AV IRT,Apr 2021,13,-10349.0,-63.4,40.836488,-73.847036,Feature,...,"[-73.847036, 40.836488]","ZEREGA AV IRT, Apr 2021: -63.4%",ZEREGA AV IRTApr 2021,5,ZEREGA AV IRT,#ff7e3d,Apr 2021,4209,Apr 2021,-63.4%


In [202]:
def color(pct_change):
    
    if pct_change > 0 : 
        return "#66cb4e"
     
    if pct_change > -10 :
        return "#a1df61"
    
    if pct_change > -20 :
        return "#c6ec85"
    
    if pct_change > -30 : 
        return "#ecfaa7"
    
    if pct_change > -40 :
        return "#fff0a6"
    
    if pct_change > -50 :
        return "#ffd280"
    
    if pct_change > -60 : 
        return "#ffb25a"
    
    if pct_change > -70 :
        return "#ff7e3d"
        
    if pct_change > -80 :
        return "#f83f24"
    
    else: 
        return "#eb0000"

# JSON

In [203]:
#Taking only the JSON variables we need from our mega dataframe.
output = df[['type', 'id', 'geometry.type', 'geometry.coordinates', 'properties.headline', 'properties.article', 'properties.radius', 'properties.name', 'properties.color', 'properties.group_id', 'properties.group_name']]
output.head()

output.to_csv("output_test.csv")

In [204]:
output['properties.group_id'] = output['properties.group_id'].map(lambda x: int(x))

<ipython-input-204-38c00b91fb1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['properties.group_id'] = output['properties.group_id'].map(lambda x: int(x))


In [205]:
ok_json = json.loads(output.to_json(orient='records'))
ok_json

[{'type': 'Feature',
  'id': 1,
  'geometry.type': 'Point',
  'geometry.coordinates': [-73.981628, 40.730953],
  'properties.headline': '1 AV BMT, March 2020: 10.9%',
  'properties.article': '1 AV BMTMarch 2020',
  'properties.radius': '5',
  'properties.name': '1 AV BMT',
  'properties.color': '#66cb4e',
  'properties.group_id': 0,
  'properties.group_name': 'March 2020'},
 {'type': 'Feature',
  'id': 2,
  'geometry.type': 'Point',
  'geometry.coordinates': [-73.981628, 40.730953],
  'properties.headline': '1 AV BMT, April 2020: -88.0%',
  'properties.article': '1 AV BMTApril 2020',
  'properties.radius': '5',
  'properties.name': '1 AV BMT',
  'properties.color': '#eb0000',
  'properties.group_id': 1,
  'properties.group_name': 'April 2020'},
 {'type': 'Feature',
  'id': 3,
  'geometry.type': 'Point',
  'geometry.coordinates': [-73.981628, 40.730953],
  'properties.headline': '1 AV BMT, May 2020: -84.4%',
  'properties.article': '1 AV BMTMay 2020',
  'properties.radius': '5',
  'prop

In [206]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [207]:
geo_format = process_to_geojson(ok_json)
geo_format

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'headline': '1 AV BMT, March 2020: 10.9%',
    'article': '1 AV BMTMarch 2020',
    'radius': '5',
    'name': '1 AV BMT',
    'color': '#66cb4e',
    'group_id': 0,
    'group_name': 'March 2020'},
   'geometry': {'type': 'Point', 'coordinates': [-73.981628, 40.730953]}},
  {'type': 'Feature',
   'properties': {'headline': '1 AV BMT, April 2020: -88.0%',
    'article': '1 AV BMTApril 2020',
    'radius': '5',
    'name': '1 AV BMT',
    'color': '#eb0000',
    'group_id': 1,
    'group_name': 'April 2020'},
   'geometry': {'type': 'Point', 'coordinates': [-73.981628, 40.730953]}},
  {'type': 'Feature',
   'properties': {'headline': '1 AV BMT, May 2020: -84.4%',
    'article': '1 AV BMTMay 2020',
    'radius': '5',
    'name': '1 AV BMT',
    'color': '#eb0000',
    'group_id': 2,
    'group_name': 'May 2020'},
   'geometry': {'type': 'Point', 'coordinates': [-73.981628, 40.730953]}},
  {'type': 'Feature'

In [208]:
#Variable name
with open('geo-data2.js', 'w') as outfile:
    outfile.write("var infoData = ")
#geojson output
with open('geo-data2.js', 'a') as outfile:
    json.dump(geo_format, outfile)

TO FIX: Get months bit to work!!!

Hopefully a slider <3

smaller radius

recenter map

In [218]:
feb20v21['change_before'].sum()

33296588.0